（1）输入一幅带人脸的图像，检测出人脸位置；

In [1]:
from cv2 import cv2

# 加载人脸分类器
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

if face_cascade.empty():
    print('Error loading face cascade')
else:
    print('Face cascade loaded successfully')


# 读取图像
img = cv2.imread('ssy.jpg')

# 转换为灰度图像
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 检测人脸
faces = face_cascade.detectMultiScale(gray, 1.3, 5)

# 绘制矩形框
for (x,y,w,h) in faces:
    cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)

# 显示图像
cv2.imshow('img',img)
cv2.waitKey(0)
cv2.destroyAllWindows()


Face cascade loaded successfully


（2）输入一幅带人脸的图像，对人脸进行识别；

In [2]:
import numpy as np

# 加载人脸检测器和识别器
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
recognizer = cv2.face.LBPHFaceRecognizer_create()

# 读取已知人脸样本及其标签
samples = []
labels = []
img = cv2.imread(f'ssy.jpg', 0)
samples.append(img)
labels.append(1)

# 训练识别器
recognizer.train(samples, np.array(labels))

# 读取测试图像
img = cv2.imread('ssy.jpg')

# 灰度化并进行人脸检测
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray, 1.3, 5)

# 遍历每个检测到的人脸并进行识别
for (x, y, w, h) in faces:
    # 提取人脸区域并进行预处理
    face_roi = gray[y:y+h, x:x+w]
    face_roi = cv2.resize(face_roi, (100, 100))

    # 进行识别
    label, confidence = recognizer.predict(face_roi)

    # 绘制人脸框及标签
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
    cv2.putText(img, f'Person {label} ({confidence:.2f})', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

# 显示结果
cv2.imshow('Result', img)
cv2.waitKey(0)
cv2.destroyAllWindows()


（3）输入一幅带人脸的图像，对人脸进行美颜；

In [3]:
# 加载美颜模型
kernel = np.ones((3,3), np.float32)/9

# 读取图像
img = cv2.imread('ssy.jpg')

cv2.imshow('Raw Face', img)

# 转换为灰度图像
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 检测人脸
faces = face_cascade.detectMultiScale(gray, 1.3, 5)

# 对每个人脸进行美颜处理
for (x,y,w,h) in faces:
    # 提取人脸区域
    face = img[y:y+h, x:x+w]

    # 对人脸区域进行美颜处理
    dst = cv2.bilateralFilter(face, 10, 150, 150)
    face = cv2.filter2D(dst, -1, kernel)


    # 将处理后的人脸区域替换回原图
    img[y:y+h, x:x+w] = face

# 显示处理后的图像
cv2.imshow('Beauty Face', img)

# 等待用户按键退出程序
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
from cv2 import cv2
import numpy as np

# 加载Haar特征分类器
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# 图像读取
img = cv2.imread('ssy.jpg')

# 将图像转为灰度图
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 检测人脸
faces = face_cascade.detectMultiScale(gray, 1.3, 5)

# 人脸美颜处理
for (x, y, w, h) in faces:
    # 截取人脸区域
    face_roi = img[y:y+h, x:x+w]

    # 图像预处理
    face_roi = cv2.GaussianBlur(face_roi, (7,7), 0)
    face_roi = cv2.cvtColor(face_roi, cv2.COLOR_BGR2YCrCb)

    # # 细节增强
    # y, cr, cb = cv2.split(face_roi)
    # y = cv2.equalizeHist(y)
    # face_roi = cv2.merge((y, cr, cb))

    # 锐化
    kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
    face_roi = cv2.filter2D(face_roi, -1, kernel)

    # 对比度调整
    alpha = 1.5
    beta = 0
    face_roi = cv2.convertScaleAbs(face_roi, alpha=alpha, beta=beta)

    raw_img = img.copy()

    img[y:y+h, x:x+w] = face_roi

    # 将处理后的图像与原始图像混合
    new_img = cv2.addWeighted(img, 0.5, raw_img, 0.5, 0)

    # 显示处理后的图像
    cv2.imshow('Beauty Face', new_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [ ]:
from cv2 import cv2
import dlib
import numpy as np

# 加载模型
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# 美颜函数
def beautify(image):
    # 灰度化
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # 人脸检测
    faces = detector(gray, 0)

    # 遍历每一张脸并进行美颜处理
    for face in faces:
        # 关键点定位
        landmarks = predictor(gray, face)

        # 获取嘴巴、左眼、右眼、鼻子和脸颊等区域的关键点坐标
        mouth = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in range(48, 68)])
        left_eye = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in range(36, 42)])
        right_eye = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in range(42, 48)])
        nose = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in range(27, 36)])
        left_cheek = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in range(0, 17)])
        right_cheek = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in range(17, 27)])

        # 对嘴巴区域进行美颜处理
        mouth_mask = np.zeros_like(gray)
        cv2.fillPoly(mouth_mask, [mouth], 255)
        mouth_mask = cv2.blur(mouth_mask, (15, 15))
        mouth_mask = cv2.GaussianBlur(mouth_mask, (15, 15), 0)
        img = cv2.seamlessClone(cv2.medianBlur(image, 15), image, mouth_mask, (int(np.mean(mouth[:, 0])), int(np.mean(mouth[:, 1]))), cv2.NORMAL_CLONE)

        # 对眼睛区域进行美颜处理
        left_eye_mask = np.zeros_like(gray)
        cv2.fillPoly(left_eye_mask, [left_eye], 255)
        left_eye_mask = cv2.blur(left_eye_mask, (15, 15))
        left_eye_mask = cv2.GaussianBlur(left_eye_mask, (15, 15), 0)
        img = cv2.seamlessClone(cv2.medianBlur(img, 15), img, left_eye_mask, (int(np.mean(left_eye[:, 0])), int(np.mean(left_eye[:, 1]))), cv2.NORMAL_CLONE)


        # 磨皮
        kernel = np.ones((3, 3), np.float32) / 9
        dst = cv2.filter2D(image, -1, kernel)
        dst = cv2.GaussianBlur(dst, (0, 0), 5)
        dst = cv2.addWeighted(image, 1.5, dst, -0.5, 0)

        # 亮度增强
        lab = cv2.cvtColor(dst, cv2.COLOR_BGR2LAB)
        l, a, b = cv2.split(lab)
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
        l = clahe.apply(l)
        lab = cv2.merge((l, a, b))
        dst = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

        # 返回美颜后的图像
        return dst

# 读取图像并进行美颜处理
img = cv2.imread("ssy.jpg")
beautified_img = beautify(img)

# 显示原图与美颜后的图像
cv2.imshow("Original Image", img)
cv2.imshow("Beautified Image", beautified_img)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [5]:
import dlib
from cv2 import cv2
import numpy as np

# 读取人脸关键点检测模型
predictor_path = 'shape_predictor_68_face_landmarks.dat'
predictor = dlib.shape_predictor(predictor_path)

# 创建人脸检测器
detector = dlib.get_frontal_face_detector()

# 定义美颜函数
def beautify_face(img):
    # 将图像转换为灰度图
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 使用人脸检测器检测人脸
    faces = detector(gray)

    # 遍历每个人脸
    for face in faces:
        # 使用关键点预测器预测人脸关键点
        landmarks = predictor(gray, face)

        # 美化嘴巴区域
        # 获取嘴巴区域的坐标
        mouth = np.array([(landmarks.part(60).x, landmarks.part(60).y),
                          (landmarks.part(61).x, landmarks.part(61).y),
                          (landmarks.part(62).x, landmarks.part(62).y),
                          (landmarks.part(63).x, landmarks.part(63).y),
                          (landmarks.part(64).x, landmarks.part(64).y),
                          (landmarks.part(65).x, landmarks.part(65).y),
                          (landmarks.part(66).x, landmarks.part(66).y),
                          (landmarks.part(67).x, landmarks.part(67).y),
                          (landmarks.part(60).x, landmarks.part(60).y)])
        # 创建一个黑色的掩膜
        mask = np.zeros(img.shape[:2], dtype=np.uint8)

        # 使用掩膜填充嘴巴区域
        cv2.fillPoly(mask, [mouth], (255, 255, 255))

        # 对嘴巴区域进行高斯模糊
        img = cv2.GaussianBlur(img, (21, 21), 0)
        img = cv2.addWeighted(img, 1.5, img, -0.5, 0, img)

        # 用掩膜来获取嘴巴区域并进行增强
        mouth_roi = img * (mask[..., np.newaxis] > 0)
        img = cv2.addWeighted(img, 1.5, mouth_roi, -0.5, 0, img)

        # # 美化脸颊区域
        # # 获取脸颊区域的坐标
        # left_cheek = np.array([(landmarks.part(1).x, landmarks.part(1).y),
        #                        (landmarks.part(2).x, landmarks.part(2).y),
        #                        (landmarks.part(3).x, landmarks.part(3).y)], np.int32)
        # right_cheek = np.array([(landmarks.part(13).x, landmarks.part(13).y),
        #                         (landmarks.part(14).x, landmarks.part(14).y),
        #                         (landmarks.part(15).x, landmarks.part(15).y)], np.int32)
        #
        # # 对脸颊区域进行高斯模糊处理
        # left_cheek_roi = img[left_cheek[0][1]:left_cheek[2][1], left_cheek[0][0]:left_cheek[2][0]]
        # right_cheek_roi = img[right_cheek[0][1]:right_cheek[2][1], right_cheek[0][0]:right_cheek[2][0]]
        # left_cheek_roi = cv2.GaussianBlur(left_cheek_roi, (7, 7), 0)
        # right_cheek_roi = cv2.GaussianBlur(right_cheek_roi, (7, 7), 0)
        #
        # # 将脸颊区域覆盖回原图
        # img[left_cheek[0][1]:left_cheek[2][1], left_cheek[0][0]:left_cheek[2][0]] = left_cheek_roi
        # img[right_cheek[0][1]:right_cheek[2][1], right_cheek[0][0]:right_cheek[2][0]] = right_cheek_roi

    return img


# 读取图像并进行美颜处理
img = cv2.imread("ssy.jpg")
beautified_img = beautify_face(img)

# 显示原图与美颜后的图像
cv2.imshow("Original Image", img)
cv2.imshow("Beautified Image", beautified_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
from cv2 import cv2
import dlib
import numpy as np

# 加载人脸检测器和关键点检测器
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

# 加载面部蒙版图像
mask = cv2.imread('mask.png', cv2.IMREAD_UNCHANGED)

# 读取图像
image = cv2.imread('face.jpg')

# 将图像转换为灰度图像
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# 检测人脸
faces = detector(gray)

# 遍历检测到的每个人脸
for face in faces:
    # 获取面部关键点坐标
    landmarks = predictor(gray, face)
    landmarks_array = np.array([[p.x, p.y] for p in landmarks.parts()])

    # 定义美颜处理的参数
    blur_factor = 5  # 模糊程度
    brightness_factor = 30  # 亮度调整程度
    contrast_factor = 1.2  # 对比度调整程度

    # 对面部区域进行美颜处理
    # 磨皮
    blurred = cv2.GaussianBlur(image, (blur_factor, blur_factor), 0)
    skin_mask = cv2.inRange(blurred, np.array([0, 58, 50]), np.array([25, 173, 230]))
    skin = cv2.bitwise_and(image, image, mask=skin_mask)
    dst = cv2.addWeighted(skin, 0.6, image, 0.4, 0)

    # 调整亮度和对比度
    dst = cv2.addWeighted(dst, contrast_factor, dst, 0, brightness_factor - 128)

    # 对蒙版进行缩放以适应面部区域大小
    mask_resized = cv2.resize(mask, (face.width(), face.height()))

    # 将面部区域的左上角坐标转换为图像坐标系中的位置
    x1, y1 = face.left(), face.top()

    # 将蒙版应用于面部区域
    mask_inv = cv2.bitwise_not(mask_resized[:, :, 3])
    mask_rgb = mask_resized[:, :, :3]
    img_roi = dst[y1:y1+face.height(), x1:x1+face.width()]
    masked = cv2.bitwise_and(img_roi, img_roi, mask=mask_inv)
    masked = cv2.add(masked, mask_rgb)

    # 将处理后的区域放回原图中
    dst[y1:y1+face.height(), x1:x1+face.width()] = masked

# 显示结果
cv2.imshow('result', dst)
cv2.waitKey(0)
cv2.destroyAllWindows()
